In [36]:
import numpy as np
import pandas as pd

In [37]:
train = pd.read_csv('./train.csv')
user_tags = pd.read_csv('./user_tags.csv')
job_tags = pd.read_csv('./job_tags.csv')
tags = pd.read_csv('./tags.csv')
test_job = pd.read_csv('../test_job.csv')

In [38]:
user_ids = np.array(list(user_tags['userID']) + list(train['userID']))
user_ids = np.unique(user_ids)
uid_idx = {}
for idx, id in enumerate(user_ids):
    uid_idx[id] = idx

In [39]:
job_ids = np.array(list(job_tags['jobID']) + list(train['jobID']))
job_ids = np.unique(job_ids)
jid_idx = {}
for idx, id in enumerate(job_ids):
    jid_idx[id] = idx

In [40]:
ratings = [list() for _ in range(user_ids.shape[0])]
for idx, row in train.iterrows():
    uidx = uid_idx[row[0]]
    jidx = jid_idx[row[1]]
    ratings[uidx].append((jidx, row[2]))

In [41]:
tag_jobs = {}
for idx, row in job_tags.iterrows():
    jidx = jid_idx[row[0]]
    if tag_jobs.get(row[1]) is None:
        tag_jobs[row[1]] = []
    tag_jobs[row[1]].append(jidx)

In [42]:
user_jobs = [list() for _ in range(user_ids.shape[0])]
for idx, row in user_tags.iterrows():
    uidx = uid_idx[row[0]]
    jobs = tag_jobs.get(row[1], [])
    for job in jobs:
        user_jobs[uidx].append(job)
for i in range(user_ids.shape[0]):
    user_jobs[i] = list(np.unique(np.array(user_jobs[i])))

In [43]:
fd = open('./data.dat', 'w')
fd.write('{} {}\n'.format(user_ids.shape[0], job_ids.shape[0]))
for rts in ratings:
    fd.write(str(len(rts)) + ' ')
    for rt in rts:
        fd.write('{} {} '.format(rt[0], rt[1]))
    fd.write('\n')
for ujs in user_jobs:
    fd.write(str(len(ujs)) + ' ')
    for uj in ujs:
        fd.write(str(uj) + ' ')
    fd.write('\n')
fd.close()

In [44]:
jobs = []
for idx, row in test_job.iterrows():
    uidx = uid_idx[row[0]]
    jidx = jid_idx[row[1]]
    jobs.append((uidx, jidx))

In [45]:
fd = open('./test.dat', 'w')
fd.write('{}\n'.format(len(jobs)))
for t in jobs:
    fd.write('{} {}\n'.format(t[0], t[1]))
fd.close()